In [1]:
import os
import sys

sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

10000


In [2]:
from pathlib import Path
SUITE2P_PATH=f"{Path.home()}/code/suite2p//"

import os
import pandas
os.environ["path"] = os.environ["path"] + r";C:\Users\jcbeg\.conda\envs\suite2p\Library\bin"
import matplotlib 
matplotlib.use('Qt5Agg')
import tifffile
class Dummy:
  ROIList = None # for sima.ROI
  tifffile = tifffile # for sima.misc
  class motion: # for sima
    MotionEstimationStrategy = object
  class speedups:
    available = False # For Shapely.speedups

def suite2PLoad():
  import sys
  import subprocess
  suite2p_path = subprocess.run(["where", "suite2p"], capture_output=True).stdout.strip()
  print("Home", Path.home())
  
  CELLPOSE_PATH=f"{Path.home()}\\.cellpose"
  if CELLPOSE_PATH not in sys.path:
    sys.path.insert(0, CELLPOSE_PATH)
  import cellpose
  sys.modules["cellpose"] = cellpose
  if suite2p_path not in sys.path:
    sys.path.insert(0, suite2p_path)
  import suite2p as run_s2p
  sys.modules["suite2p"] = run_s2p
  print("Done")
  return run_s2p

run_s2p = suite2PLoad()

Home C:\Users\jcbeg
creating new log file
2021-12-09 16:18:08,145 [INFO] WRITING LOG OUTPUT TO C:\Users\jcbeg\.cellpose\run.log
Done


In [8]:
BASE_DIR = Path(r"C:\Users\jcbeg\Imaging\SPARC\20210809\\")

import os
def getFilesWithExt(top_dir, files_ext):
  matches = []
  for root, dirnames, filenames in os.walk(str(top_dir)):
    for _dir in dirnames:
      matches += getFilesWithExt(_dir, files_ext)
    for filename in filenames:
      full_path = os.path.join(root, filename)
      if full_path.endswith(files_ext):
        matches.append(Path(os.path.join(root, filename)))
  return matches 


In [ ]:
from pathlib import Path

def convertND2ToTiff(fp_pathlib):
  import numpy as np
  from PIL import Image
  from nd2reader import ND2Reader
  import matplotlib.pyplot as plt
  print("Attempting to covert:", str(fp_pathlib))
  save_fp = tiffPathFromND2(fp_pathlib)
  save_dir = save_fp.parent
  print(f"Saving to: {save_fp} in dir: {save_dir}")
  save_dir.mkdir(parents=False, exist_ok=True) # For now, doesn't create except one-level dir
  with ND2Reader(str(fp_pathlib)) as images:
    images_li = []
    images.iter_axes = 't' #Possible parameter options = 'c' 't' 'v' 'x' 'y' 'z'
    for idx in range(len(images)):
      images_li.append(Image.fromarray(np.array(images[idx])))
    images_li[0].save(save_fp, save_all=True, append_images=images_li[1:])
    print("Done converting")

def tiffPathFromND2(_file):
  return Path(f"{_file.parent}/{_file.stem}/{_file.stem}.tiff")

def iterConvert():
  #fp = r'C:\Users\jcbeg\Imaging\pt_antibodies - Patient Antibodies CE 200k base 001\WT RAT C060820 DIV30\001 Control\well_002.nd2'
  tiff_files = getFilesWithExt(BASE_DIR, ".tiff")
  files_to_convert = [_file for _file in getFilesWithExt(BASE_DIR, ".nd2")
                     if tiffPathFromND2(_file) not in tiff_files]
  print("Files to convert:", files_to_convert)
  print("Total number of files to convert:", len(files_to_convert))
  import tqdm
  for fp in tqdm.tqdm(files_to_convert):
    print(f"Processing {fp.name}.tiff")
    convertND2ToTiff(fp) 
iterConvert()

In [5]:
#%matplotlib widget 
# notebook

import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

def load(fp):
  stat = np.load(f"{fp}stat.npy", allow_pickle=True)
  ops = np.load(f'{fp}ops.npy', allow_pickle=True).item()
  F = np.load(f"{fp}F.npy", allow_pickle=True)
  Fneu = np.load(f"{fp}F.npy", allow_pickle=True)
  spks = np.load(f"{fp}spks.npy", allow_pickle=True)
  #iscell = np.load('iscell.npy', allow_pickle=True)
  return stat, ops, F, Fneu, spks # iscell

#def iscell(ops)
  #iscell = ops["iscell]"]


def getMeanImg(ops):
  meanImg = ops["meanImg"] # Also "max_proj", "meanImg", "meanImgE"
  mimg = meanImg # Use suite-2p source-code naming
  mimg1 = np.percentile(mimg,1)
  mimg99 = np.percentile(mimg,99)
  mimg = (mimg - mimg1) / (mimg99 - mimg1)
  mimg = np.maximum(0,np.minimum(1,mimg))
  mimg *= 255
  mimg = mimg.astype(np.uint8)
  return mimg


def getStats(stat, frame_shape):
  MIN_SKEW = 1.5 #TODO: define for iscell; also figure exclude noncell
  pixel2neuron = np.full(frame_shape, fill_value=np.nan, dtype=np.float)
  scatters = dict(x=[], y=[], color=[], text=[])
  min_skew, max_skew = -20000, 20000 #swapoped min/max 20,000 *-1
  nid2idx = {}
  nid2idx_rejected = {}
  print(f"Num cells: {stat.shape[0]}")
  from suite2p.gui.utils import boundary
  for n in range(stat.shape[0]):
    skew = stat[n]['skew']
    min_skew, max_skew = min(skew, min_skew), max(skew, max_skew)
    if skew > MIN_SKEW:
      nid2idx[n] = len(scatters["x"]) # Assign new idx
    else:
      nid2idx_rejected[n] = len(scatters["x"])
    ypix = stat[n]['ypix'].flatten() - 1#[~stat[n]['overlap']] - 1
    xpix = stat[n]['xpix'].flatten() - 1#[~stat[n]['overlap']] - 1
    yext, xext = boundary(ypix, xpix)
    scatters['x'] += [xext]
    scatters['y'] += [yext]
    pixel2neuron[xpix, ypix] = n
    scatters["color"].append(skew)
    scatters["text"].append(f"Cell #{n} - Skew: {skew}")
  print("Min/max skew: ", min_skew, max_skew)
  # Normalize colors between 0 and 1
  color_raw = np.array(scatters["color"])
  scatters["color"] = (color_raw - min_skew) / (max_skew - min_skew)
  return scatters, nid2idx, nid2idx_rejected, pixel2neuron

def spkTrace(neuron_id, spks):
  trace = spks[neuron_id]
  trace = (trace - trace.min()) / (trace.max() - trace.min())
  #print(trace)
  trace[trace < .3*trace.max()] = 0
  spk_times = np.where(trace > 0)[0]
  last_spk_time = spk_times[0] 
  for cur_spk_time in spk_times[1:]:
    if cur_spk_time - last_spk_time < 20:
      trace[cur_spk_time] = 0
      #print(f"Removed spike at {cur_spk_time} because - {last_spk_time}")
    else:
      last_spk_time = cur_spk_time
  return trace

def dispPlot(meanImg, scatters, nid2idx, nid2idx_rejected,
             pixel2neuron, F, Fneu, spks, axs=None):
  if axs is None:
    fig = plt.figure(constrained_layout=True)
    NUM_GRIDS=12
    gs = fig.add_gridspec(NUM_GRIDS, 1)
    ax1 = fig.add_subplot(gs[:NUM_GRIDS-2])
    ax2 = fig.add_subplot(gs[NUM_GRIDS-2])
    fig.set_size_inches(6,7)
  else:
    ax1 = axs
    ax1.set_xlim(0, meanImg.shape[0])
    ax1.set_ylim(meanImg.shape[1], 0)
    ax2 = None
  ax1.imshow(meanImg, cmap='gray')
  #fig.update_xaxes(showticklabels=False)
  #fig.update_yaxes(showticklabels=False)
  print("Neurons count:", len(nid2idx))
  norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True)
  mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)
  #print(mapper.to_rgba(scatters['skew']))
  def plotDict(n2d2idx_dict, override_color=None):
    for neuron_id, idx in n2d2idx_dict.items():
      color = override_color if override_color else mapper.to_rgba(scatters['color'][idx])
      #print(f"{idx}: {scatters['x'][idx]} - {scatters['y'][idx]}")
      sc = ax1.scatter(scatters["x"][idx], scatters['y'][idx], color=color,
                      marker=".", s=1)
  plotDict(nid2idx)
  plotDict(nid2idx_rejected, 'k')
  ax1.set_title(f"Neurons Used: {len(nid2idx)}/{len(nid2idx)+len(nid2idx_rejected)} detected (black)")
  if ax2 is None:
    return
  nan_fills = np.full(F.shape[1], fill_value=np.nan)
  neuron_trace_plot = ax2.plot(np.arange(F.shape[1]), nan_fills, color='blue')[0]
  neuron_trace_plot2 = ax2.plot(np.arange(F.shape[1]), nan_fills, color='red')[0]
  text=ax1.text(0,1.06, "No neuron selected")
  print(f"spks.shape: {spks.dtype}")
  twinx = ax2.twinx()
  spks_trace_plot = twinx.plot(np.arange(spks.shape[1]), nan_fills,
                               color="gray")[0]
  twinx.set_ylim(0, 1)
  # ax2.set_autoscaley_on(True)n seleced", va="top", ha="left", transform=ax1.transAxes)
  def onclick(event):
    neuron_id = pixel2neuron[int(event.xdata), int(event.ydata)]
    tx = f"x={event.xdata:.2f} - y={event.ydata:.2f} - Pixel: {neuron_id}"
    if np.isnan(neuron_id):
      tx = "Not a neuron"
    else:
      pass
      neuron_id = int(neuron_id)
      idx = nid2idx[neuron_id]
      #tx = f"Neuron: {neuron_id} - Skew: {scatters['skew'][idx]:.2f}"
      tx = scatters['text'][idx]
      F_n = F[neuron_id]
      spikes = spkTrace(neuron_id, spks)
      # https://stackoverflow.com/a/24272092/11996983
      neuron_trace_plot.set_ydata(F_n)
      #from scipy.ndimage import filters
      #F_n = F_n - filters.minimum_filter1d(F_n, 10) + min_
      F_n2 = (F_n - 0.7*Fneu[neuron_id])
      F_n2 += F_n.min() - F_n2.min()
      neuron_trace_plot2.set_ydata(F_n2)
      spks_trace_plot.set_ydata(spikes)
      ax2.relim()
      ax2.autoscale_view()
      fig.canvas.draw()
      fig.canvas.flush_events()
    text.set_text(tx)
    return True
  cid = fig.canvas.mpl_connect('button_press_event', onclick)
  
def show(fp):
  stat, ops, F, Fneu, spks = load(fp)
  meanImg = getMeanImg(ops)
  scatters, nid2idx, nid2idx_rejected, pixel2neuron = getStats(stat, meanImg.shape)
  #dispPlot(meanImg, scatters, nid2idx, nid2idx_rejected, pixel2neuron, F, Fneu, spks)

In [10]:
#show(PATH)
%matplotlib widget

def processAll():
  count = 0
  for fp in getFilesWithExt(BASE_DIR, "spks.npy"):
    parent_dir = str(fp.parent) + os.path.sep
    print(parent_dir)
    show(parent_dir)
    count += 1
    if count == 1:
      break
  # for fp in PATH_GBZ[0:2]:
  #   print(fp)
  #   show(fp)
processAll()

In [ ]:
def neuronID(nid2idx): # Used for debugging
  idxs = [6, 50, 78]
  nid2idx_keys = list(nid2idx.keys())
  for idx in idxs:
    key = nid2idx_keys[idx]
    print("Idx:", idx, "=", key)

def createGrid():
  fig = plt.figure(constrained_layout=True)
  fig.set_size_inches(12, 5)
  gs = fig.add_gridspec(2, 10)
  ax_img = fig.add_subplot(gs[:, 6:])
  ax_psth = fig.add_subplot(gs[0, :3])
  ax_sync = fig.add_subplot(gs[1, :3])
  ax_fr = fig.add_subplot(gs[0, 3:6])
  ax_cvr = fig.add_subplot(gs[1, 3:6])
  return fig, (ax_psth, ax_sync, ax_fr, ax_cvr, ax_img)

def process(fp, bin_size, last_fig):
  stat, ops, F, Fneu, spks = load(fp)
  scatters, nid2idx, nid2idx_rejected, pixel2neuron = getStats(stat, (ops["Lx"], ops["Ly"]))
  #spikes = np.zeros((len(nid2idx), spks.shape[1]), dtype=np.bool)
  # neuronID(nid2idx)
  spikes_count = []
  neurons_cv = []
  bins = np.arange(0, spks.shape[1] + 1, bin_size)
  num_neurons = len(nid2idx)
  spikes_binned = np.zeros([num_neurons, len(bins)+1], dtype=np.bool)
  #print(bins)
  csv_lines = {-1:'NeuronId,IsUsed,Skew,SpikesTimes'}
  for idx, neuron_id in enumerate(nid2idx.keys()):
    spike_times = np.where(spkTrace(neuron_id, spks))[0]
    #spikes[idx, spike_times] = 1
    spikes_count.append(len(spike_times))
    neuron_binned = np.digitize(spike_times, bins)
    #print(neuron_binned)
    spikes_binned[idx,neuron_binned] = 1
    isi = np.diff(spike_times)
    neurons_cv.append(isi.std()/isi.mean())
    csv_lines[neuron_id] = f'{neuron_id},True,{stat[neuron_id]["skew"]},'+\
                           f'"{str(list(spike_times)).lstrip("[").rstrip("]")}"'
  for neuron_id in nid2idx_rejected.keys():
    spike_times = np.where(spkTrace(neuron_id, spks))[0]
    csv_lines[neuron_id] = f'{neuron_id},False,{stat[neuron_id]["skew"]},'+\
                           f'"{str(list(spike_times)).lstrip("[").rstrip("]")}"'
  import matplotlib.pyplot as plt
  if last_fig:
    plt.close(last_fig)
  #fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1)
  fig, (ax_psth, ax_sync, ax_fr, ax_cvr, ax_img) = createGrid()
  last_fig = fig
  NUM_PARENTS = 3
  _dir = Path(fp).parent.parent
  title = str(_dir.name)
  for i in range(NUM_PARENTS):
    _dir = _dir.parent
    title = f"{_dir.name}/{title}"
  fig.suptitle(title, fontsize='x-small')
  meanImg = getMeanImg(ops)
  dispPlot(meanImg, scatters, nid2idx, nid2idx_rejected, pixel2neuron,
           F, Fneu, spks, axs=ax_img)
  ax_psth.set_title("PSTH", fontsize='x-small')
  ax_psth.margins(x=0, y=0)
  #ax_psth.eventplot(spikes)
  ax_psth.imshow(spikes_binned, cmap='binary')
  ax_psth.set_xlabel(f"Time bin (bin-size = {bin_size} frames, {bin_size/ops['fs']} Sec)")
  ax_psth.set_ylabel("Neuron ID")
  #mean_firing_rate = np.log(spikes_binned.mean().mean())
  spikes_count = np.array(spikes_count)
  # print(f"Max time: {spks.shape[0]}")
  # print(f"Max spike time: {spikes_count.max()}")
  firing_rate = -1/np.log(spikes_count/(spks.shape[0]*ops["fs"]))
  # print("Firing rate:", firing_rate)
  mean_firing_rate = firing_rate.mean()
  median_firing_rate = np.median(firing_rate)
  ax_fr.hist(firing_rate, color='purple')
  ax_fr.set_ylabel(f"Neurons count", fontsize='x-small')
  ax_fr.set_xlabel(f"-1/log(Mean first-spike firing rate) "
                   f"(mean={mean_firing_rate:.4f}, median={median_firing_rate:.4f})", fontsize='x-small')
  ax_fr.set_title("First-spike firing rate", fontsize='x-small')
  # plot cv
  neurons_cv = np.array(neurons_cv)
  ax_cvr.hist(neurons_cv, color='g')
  ax_cvr.set_title("Neurons CV (coefficient of variation)", fontsize='x-small')
  ax_cvr.set_ylabel(f"Neurons count", fontsize='x-small')
  ax_cvr.set_xlabel(f"CV (mean={np.nanmean(neurons_cv)}, median={np.nanmedian(neurons_cv)})", fontsize='x-small')
  spike_count_per_neuron = spikes_binned.sum(axis=1)
  sums = spikes_binned.sum(axis=0)
  #print(f"Spiked bins: {spikes_binned.shape} - sums: {sums.shape}")
  #print(f"sums: {sums}")
  RANGE_STEP=1
  bins_range = np.arange(0, spikes_binned.shape[0] + 1, RANGE_STEP)
  # binned = np.histogram(sums, bins=bins_range) # Returned bins_range is the same
  # binned, bins_range = binned # It's already a tuple
  # print(f"Binned: {binned.shape} - range: {bins_range.shape}")
  OFFSET = 3 # Skip first few sums as they destroying the view
  ax_sync.hist(sums, bins=bins_range[OFFSET:], color='r')
  ax_sync.set_xlabel(f"Num. Neurons in Synchronized firing (skipped events < {OFFSET})", fontsize='x-small')
  ax_sync.set_ylabel("Synchronized firing count", fontsize='x-small')
  # fig.tight_layout()
  save_fp = f"{Path(fp).parent.parent.parent}/{Path(fp).parent.parent.name}"
  fig.savefig(f"{save_fp}.png")
  with open(f"{save_fp}.csv", 'w') as f:
    for neuron_id in sorted(csv_lines.keys()):
      f.write(f"{csv_lines[neuron_id]}\n")
  return last_fig

Error: Session cannot generate requests

In [ ]:
if 'LAST_FIGS1' not in locals():
    LAST_FIGS1 = [None]

def processFigs():
  global LAST_FIGS1
  if True:
    import matplotlib.pyplot as plt
    plt.close('all')
  _files = getFilesWithExt(BASE_DIR, "spks.npy")
  if len(LAST_FIGS1) < len(_files):
    LAST_FIGS1 = [None]*len(_files)
  for idx, fp in enumerate(_files):
    _dir = str(fp.parent) + os.path.sep
    print(_dir)
    if idx == 2: break
    LAST_FIGS1[idx] = process(_dir, bin_size=5, last_fig=LAST_FIGS1[idx])

processFigs()

In [ ]:
# if 'LAST_FIGS2' not in locals():
#     LAST_FIGS2 = [None]* len(PATH_GBZ)
# for idx, fp in enumerate(PATH_GBZ[0:4]):
#     print(fp)
#     LAST_FIGS2[idx] = process(fp, bin_size=5, last_fig=LAST_FIGS2[idx])